In [3]:
import pandas as pd
import tensorflow as tf
import numpy as np
import io 
from PIL import Image
from sklearn.model_selection import train_test_split

In [6]:
# import dati
DF_FEM = pd.read_parquet("hf://datasets/flwrlabs/femnist/data/train-00000-of-00001.parquet")
DF_WRITERS = DF_FEM["writer_id"].unique()

# ID_DF_FEM = DF_FEM[DF_FEM["writer_id"] == DF_WRITERS[id]]
images = []
for img_dict in DF_FEM['image']:
    img_bytes = img_dict['bytes']
    img = Image.open(io.BytesIO(img_bytes)).convert('L')    # 'L' per convertire in scala di grigi
    img_array = np.array(img).reshape(28, 28, 1)
    images.append(img_array)
images = np.array(images).astype(np.float32) / 255.0        # Normalizza le immagini (da 0 a 1)
X = images
Y = DF_FEM.iloc[:, 3]

In [7]:
print(X.shape)
print(Y.shape)

(814277, 28, 28, 1)
(814277,)


---

In [8]:
# featmaps = [32, 64, 128]                                        # Example values for feature maps (number of filters)
# kernels = [3, 3, 3]                                             # Example kernel sizes
# first_linear_size = featmaps[2] * kernels[2] * kernels[2]       # Example size after flattening (depends on input size and feature maps)
# linears = [512, 256, 62]                                        # Example sizes of the fully connected layers
# num_classes = 62                                                # Number of classes in FEMNIST

In [9]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Input(shape=(28, 28, 1)),
#     tf.keras.layers.Conv2D(filters=featmaps[0], kernel_size=kernels[0], padding='same', activation='relu'),
#     tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='valid'),
#     tf.keras.layers.Conv2D(filters=featmaps[1], kernel_size=kernels[1], padding='same', activation='relu'),
#     tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='valid'),
#     tf.keras.layers.Conv2D(filters=featmaps[2], kernel_size=kernels[2], padding='same', activation='relu'),
#     tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='valid'),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(linears[0], activation='relu'),
#     tf.keras.layers.Dense(linears[1]),
#     tf.keras.layers.Dense(num_classes, activation='log_softmax'),
# ])
# model.compile("adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(62, activation="softmax"),
])
model.compile("adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [10]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.3)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=42, test_size=0.3)

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(tf.data.AUTOTUNE)

In [11]:
print(len(train_dataset))
print(len(test_dataset))
print(len(val_dataset))

12469
7634
5344


Using tf.data.Dataset.from_tensor_slices

In [12]:
history = model.fit(train_dataset, epochs=20, batch_size=32, verbose=0, validation_data=val_dataset)

In [13]:
loss, accuracy = model.evaluate(test_dataset, verbose=0)
print(accuracy)

0.8628113269805908


aggiugnere validation set-

In [14]:
y_pred = model.predict(x_test, verbose=0)

In [15]:
from sklearn.metrics import accuracy_score

# y_test = y_test.to_numpy()
print(y_test)
y_pred = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred)

568345    43
102796    57
346529     1
34036     36
286429    38
          ..
434331    24
569218    23
569754    43
17337     27
32461     14
Name: character, Length: 244284, dtype: int64


In [16]:
np.unique(y_pred)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61])

In [17]:
print(accuracy)

0.8628113179741612


In [18]:
loss

0.4042328894138336